In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
np.random.seed(7)

In [3]:
# load dataset
dataframe = pd.read_csv("pca_75_clas.csv", sep=',')

In [4]:
dataframe.head(3)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,priceUSD
0,0.074162,0.015329,-0.048046,0.042709,0.007321,-0.014251,0.001355,-0.044263,-0.014403,-0.036199,...,0.017701,-0.020600,-0.021125,-0.001148,-0.004502,-0.012360,-0.032049,0.007081,0.006557,1
1,0.094841,0.072671,-0.077840,-0.014523,0.027039,-0.053013,0.056817,-0.009060,0.047423,-0.009912,...,-0.047544,0.013065,0.065670,0.006482,0.020321,0.007130,0.016320,0.013705,-0.042491,1
2,0.064880,0.028643,-0.038454,0.019065,0.028725,-0.014173,-0.002313,-0.031474,-0.009467,-0.034115,...,0.020285,0.006481,-0.012896,0.008115,-0.022120,-0.021993,0.012241,0.021045,-0.033730,1


In [5]:
dataframe.shape

(735, 51)

In [6]:
length=dataframe.shape[1]-1

In [7]:
length

50

In [9]:
# split into input (X) and output (Y) variables
X = dataframe.iloc[:,0:length]
y = dataframe['priceUSD']

In [10]:
X.head(3)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.074162,0.015329,-0.048046,0.042709,0.007321,-0.014251,0.001355,-0.044263,-0.014403,-0.036199,...,-0.004087,0.017701,-0.020600,-0.021125,-0.001148,-0.004502,-0.012360,-0.032049,0.007081,0.006557
1,0.094841,0.072671,-0.077840,-0.014523,0.027039,-0.053013,0.056817,-0.009060,0.047423,-0.009912,...,0.003421,-0.047544,0.013065,0.065670,0.006482,0.020321,0.007130,0.016320,0.013705,-0.042491
2,0.064880,0.028643,-0.038454,0.019065,0.028725,-0.014173,-0.002313,-0.031474,-0.009467,-0.034115,...,0.014521,0.020285,0.006481,-0.012896,0.008115,-0.022120,-0.021993,0.012241,0.021045,-0.033730


In [11]:
y=np.ravel(y)

In [12]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,

In [13]:
shape=X.shape[1]

In [14]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8, shuffle=False, random_state=7)

In [15]:
estimators=[]

In [16]:
estimators.append(['robust',RobustScaler()])

In [17]:
estimators.append(['mixmax',MinMaxScaler()])

In [18]:
scale=Pipeline(estimators,verbose=True)

In [19]:
scale.fit(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


Pipeline(memory=None,
         steps=[('robust',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ['mixmax', MinMaxScaler(copy=True, feature_range=(0, 1))]],
         verbose=True)

In [20]:
X_train=scale.transform(X_train)

In [21]:
X_test=scale.transform(X_test)

In [22]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-2
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [23]:
# define neural network model
def sequential_model(initializer='normal', activation='relu', neurons=300, NUM_FEATURES=shape):
    # create model
    model = Sequential()
    model.add(Dense(400, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
    model.add(Dense(500, activation=activation))
    model.add(Dense(100, activation=activation))
    model.add(Dense(2, activation='softmax', kernel_initializer=initializer))
    # Compile model
    adam=Adam(lr=lr_schedule(0),amsgrad=True)
    #sgd=keras.optimizers.SGD(learning_rate=0.08, momentum=0.9, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [24]:
mcp_save = ModelCheckpoint('trained_models/ANN_cls_interval3_pca.hdf5', save_best_only=True, monitor='val_loss', mode='max')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100,verbose=1, mode='max')

In [27]:
classifier=KerasClassifier(build_fn=sequential_model,batch_size=32, epochs=1000,validation_split=0.1,validation_freq=1, shuffle=True,use_multiprocessing=True, callbacks=[mcp_save,earlyStopping])

In [28]:
classifier.fit(X_train,y_train)

Learning rate:  0.01
Train on 529 samples, validate on 59 samples
Epoch 1/1000
529/529 [==============================] - 0s 514us/step - loss: 0.7168 - accuracy: 0.5236 - val_loss: 0.6990 - val_accuracy: 0.4576
Epoch 2/1000
529/529 [==============================] - 0s 227us/step - loss: 0.6927 - accuracy: 0.5369 - val_loss: 0.7017 - val_accuracy: 0.4576
Epoch 3/1000
529/529 [==============================] - 0s 224us/step - loss: 0.6921 - accuracy: 0.5331 - val_loss: 0.6998 - val_accuracy: 0.4576
Epoch 4/1000
529/529 [==============================] - 0s 220us/step - loss: 0.6911 - accuracy: 0.5331 - val_loss: 0.6999 - val_accuracy: 0.4576
Epoch 5/1000
529/529 [==============================] - 0s 238us/step - loss: 0.6915 - accuracy: 0.5331 - val_loss: 0.6988 - val_accuracy: 0.4576
Epoch 6/1000
529/529 [==============================] - 0s 239us/step - loss: 0.6915 - accuracy: 0.5331 - val_loss: 0.6994 - val_accuracy: 0.4576
Epoch 7/1000
529/529 [==============================] - 0s

529/529 [==============================] - 0s 250us/step - loss: 0.6912 - accuracy: 0.5331 - val_loss: 0.7001 - val_accuracy: 0.4576
Epoch 57/1000
529/529 [==============================] - 0s 213us/step - loss: 0.6913 - accuracy: 0.5331 - val_loss: 0.7026 - val_accuracy: 0.4576
Epoch 58/1000
529/529 [==============================] - 0s 224us/step - loss: 0.6910 - accuracy: 0.5331 - val_loss: 0.7014 - val_accuracy: 0.4576
Epoch 59/1000
529/529 [==============================] - 0s 213us/step - loss: 0.6912 - accuracy: 0.5331 - val_loss: 0.6999 - val_accuracy: 0.4576
Epoch 60/1000
529/529 [==============================] - 0s 209us/step - loss: 0.6913 - accuracy: 0.5331 - val_loss: 0.6998 - val_accuracy: 0.4576
Epoch 61/1000
529/529 [==============================] - 0s 216us/step - loss: 0.6911 - accuracy: 0.5331 - val_loss: 0.7003 - val_accuracy: 0.4576
Epoch 62/1000
529/529 [==============================] - 0s 234us/step - loss: 0.6912 - accuracy: 0.5331 - val_loss: 0.7014 - val_ac

Epoch 112/1000
529/529 [==============================] - 0s 255us/step - loss: 0.6914 - accuracy: 0.5331 - val_loss: 0.7027 - val_accuracy: 0.4576
Epoch 113/1000
529/529 [==============================] - 0s 248us/step - loss: 0.6922 - accuracy: 0.5331 - val_loss: 0.7051 - val_accuracy: 0.4576
Epoch 114/1000
529/529 [==============================] - 0s 236us/step - loss: 0.6913 - accuracy: 0.5331 - val_loss: 0.7003 - val_accuracy: 0.4576
Epoch 115/1000
529/529 [==============================] - 0s 242us/step - loss: 0.6910 - accuracy: 0.5331 - val_loss: 0.6989 - val_accuracy: 0.4576
Epoch 116/1000
529/529 [==============================] - 0s 229us/step - loss: 0.6912 - accuracy: 0.5331 - val_loss: 0.6988 - val_accuracy: 0.4576
Epoch 117/1000
529/529 [==============================] - 0s 256us/step - loss: 0.6910 - accuracy: 0.5331 - val_loss: 0.7004 - val_accuracy: 0.4576
Epoch 118/1000
529/529 [==============================] - 0s 271us/step - loss: 0.6912 - accuracy: 0.5331 - val_

529/529 [==============================] - 0s 249us/step - loss: 0.6922 - accuracy: 0.5331 - val_loss: 0.7042 - val_accuracy: 0.4576
Epoch 168/1000
529/529 [==============================] - 0s 256us/step - loss: 0.6914 - accuracy: 0.5331 - val_loss: 0.6998 - val_accuracy: 0.4576
Epoch 169/1000
529/529 [==============================] - 0s 248us/step - loss: 0.6911 - accuracy: 0.5331 - val_loss: 0.7004 - val_accuracy: 0.4576
Epoch 170/1000
529/529 [==============================] - 0s 213us/step - loss: 0.6911 - accuracy: 0.5331 - val_loss: 0.7017 - val_accuracy: 0.4576
Epoch 171/1000
529/529 [==============================] - 0s 256us/step - loss: 0.6911 - accuracy: 0.5331 - val_loss: 0.7026 - val_accuracy: 0.4576
Epoch 172/1000
529/529 [==============================] - 0s 254us/step - loss: 0.6911 - accuracy: 0.5331 - val_loss: 0.7031 - val_accuracy: 0.4576
Epoch 173/1000
529/529 [==============================] - 0s 229us/step - loss: 0.6917 - accuracy: 0.5331 - val_loss: 0.7000 - 

529/529 [==============================] - 0s 244us/step - loss: 0.6913 - accuracy: 0.5331 - val_loss: 0.7014 - val_accuracy: 0.4576
Epoch 223/1000
529/529 [==============================] - 0s 246us/step - loss: 0.6912 - accuracy: 0.5331 - val_loss: 0.7015 - val_accuracy: 0.4576
Epoch 224/1000
529/529 [==============================] - 0s 246us/step - loss: 0.6909 - accuracy: 0.5331 - val_loss: 0.6998 - val_accuracy: 0.4576
Epoch 225/1000
529/529 [==============================] - 0s 233us/step - loss: 0.6914 - accuracy: 0.5331 - val_loss: 0.7020 - val_accuracy: 0.4576
Epoch 226/1000
529/529 [==============================] - 0s 219us/step - loss: 0.6912 - accuracy: 0.5331 - val_loss: 0.6997 - val_accuracy: 0.4576
Epoch 00226: early stopping


In [31]:
prediction_model = load_model('trained_models/ANN_cls_interval3_pca.hdf5',compile=False)

In [32]:
y_pred = prediction_model.predict_classes(X_test)

In [33]:
acc=accuracy_score(y_test,y_pred)
acc

0.48299319727891155

In [34]:
f1=f1_score(y_test,y_pred,average='weighted')
f1

0.3146102477688323

In [30]:
auc=roc_auc_score(y_test,y_pred)
auc

0.5254556779438762

In [31]:
y_prob=[prediction_model.predict(X_test).max() for i in range(len(y_test))]

In [32]:
print(classification_report(y_test,y_pred,labels=[0,1], target_names=['decrease','increase']))

              precision    recall  f1-score   support

    decrease       0.54      0.31      0.40       246
    increase       0.52      0.74      0.61       248

    accuracy                           0.53       494
   macro avg       0.53      0.53      0.50       494
weighted avg       0.53      0.53      0.50       494



In [33]:
predictions=pd.DataFrame(zip(np.ravel(y_test),np.ravel(y_pred)),columns=['y_test','y_pred'])

In [34]:
predictions

,y_test,y_pred
0,0,0
1,1,0
2,1,0
3,1,0
4,0,1
5,1,0
6,1,0
7,1,1
8,1,1
9,1,0
